In [ ]:
from datasets import load_dataset, Dataset
import numpy as np
from utils import channel_4_to_3
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Rescaling
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
import tensorflow as tf
import cv2

from utils import read_class_names

%load_ext autoreload
%autoreload 2

This code references the Tensorflow Image Classification tutorial that was found at this link https://www.tensorflow.org/tutorials/images/classification.

NOTE: Outputs are not shown due to memory limitations on Github.

In [ ]:
# DEFINE CONSTANTS
batch_size = 64
test_size = 0.2
img_height = 256
img_width = 256

In [ ]:
# import datasets

# ingredients = load_dataset("mrdbourke/food_vision_199_classes")
fruit_veg = load_dataset("arjunpatel1234/Recipe_Ingredients")
fruit_veg

In [ ]:
classes = read_class_names("../dataset/recipe_ingredient_names.txt")

In [ ]:
cleaned = fruit_veg.map(channel_4_to_3)

In [ ]:
cleaned = cleaned['train'].train_test_split(test_size=test_size)
print(cleaned)
food_train = cleaned['train'].to_tf_dataset(
                columns=["image"],
                label_cols=["label"],
                batch_size=batch_size,
                shuffle=True
            )
food_test = cleaned['test'].to_tf_dataset(
                columns=["image"],
                label_cols=["label"],
                batch_size=batch_size,
                shuffle=False
            )

In [ ]:
import matplotlib.pyplot as plt

class_names = list(classes.values())
plt.figure(figsize=(10, 10))

for images, labels in food_train.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = food_train.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = food_test.cache().prefetch(buffer_size=AUTOTUNE)

normalization_layer = Rescaling(1./255)
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
noramlized_ds = normalized_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))


In [ ]:
num_classes = len(class_names)

model = Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
import os

checkpoint_path = "cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
epochs=15
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
  callbacks=[cp_callback]
)

In [ ]:
model.save("model_basic.keras")

In [ ]:
model = tf.keras.models.load_model('model_basic.keras')

In [ ]:
x_test, y_test = [], []

for images, labels in food_test:
  for i in range(len(images)):
    x_test.append(images.numpy()[i])
    y_test.append(labels.numpy()[i])

x_test = np.array(x_test)
y_test = np.array(y_test)
y_test

In [ ]:
test_scores = model.evaluate(x_test, y_test, verbose=2)
print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1])

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
data_augmentation = Sequential(
  [
    layers.RandomFlip("horizontal",
                      input_shape=(img_height,
                                  img_width,
                                  3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ]
)

In [ ]:
plt.figure(figsize=(10, 10))
for images, _ in food_train.take(1):
  for i in range(9):
    augmented_images = data_augmentation(images)
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(augmented_images[0].numpy().astype("uint8"))
    plt.axis("off")

In [ ]:
model_augmented = Sequential([
  data_augmentation,
  layers.Rescaling(1./255),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes, name="outputs")
])

model_augmented.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
checkpoint_path = "cp_augmented.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
epochs=15
history = model_augmented.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
  callbacks=[cp_callback]
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
test_scores = model_augmented.evaluate(x_test, y_test, verbose=2)
print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1])

In [ ]:
model_augmented.save("model_augmented.keras")

In [ ]:
model_augmented = tf.keras.models.load_model('model_augmented.keras')